In [7]:
import numpy as np
import pandas as pd
import os

In [8]:
data_directory = "../Data/Pilot/PS14/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [9]:
def clean_df(df):
    result = df[df['Movement'] != "start"]
    result = result[result['Movement'] != "transition"]
    result = result[result['EyeTrackingDataValid'] != 'TRUE']
    return result

In [24]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.arccos(np.dot([row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']], [row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z']])/(np.linalg.norm([row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']]) * np.linalg.norm([row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z']]))))
    return df

In [10]:
def calculate_euclidean_error(df):
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [22]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.mkdir(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [25]:
calculate_errors("error")

KeyError: 'EyeGazeDirection.x'